# Comparing 3 traditional classifiers

In this notebook I will compare the performance of 3  
traditional ML classifiers  
* Logistic Regression
* Gradient Boosting
* Random Forests

Perhaps in a future notebook I will include a Neural Net, but not here.


## Data set
The data set has 9 features on stroke patients and 
the target variable is whether the patient had a stroke.
The data set can be found on Kaggle.


In [79]:
import os
import datetime
import pandas as pd
import numpy as np
from IPython.display import display
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import log_loss

In [80]:
orig_df = pd.read_csv("full_data.csv")

In [81]:
display(orig_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4981 entries, 0 to 4980
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   gender             4981 non-null   object 
 1   age                4981 non-null   float64
 2   hypertension       4981 non-null   int64  
 3   heart_disease      4981 non-null   int64  
 4   ever_married       4981 non-null   object 
 5   work_type          4981 non-null   object 
 6   Residence_type     4981 non-null   object 
 7   avg_glucose_level  4981 non-null   float64
 8   bmi                4981 non-null   float64
 9   smoking_status     4981 non-null   object 
 10  stroke             4981 non-null   int64  
dtypes: float64(3), int64(3), object(5)
memory usage: 428.2+ KB


None

## Categorical variables
Note that the data set has categorical variales.  
I will used Pandas to do one hot encoding of these variables.

In [82]:
# one hot encoding of the categorical variables
#  remove the originals from the new dataframe
df = orig_df.copy()
cat_cols = [idx for idx, val in df.dtypes.iteritems() if val == "object"]
dummies = pd.get_dummies(df[cat_cols], drop_first=True)
one_hot_cols = list(dummies.columns)
df = pd.concat([orig_df, dummies], axis=1)
df.drop(labels=cat_cols, axis=1, inplace=True)
df.head(2)

,age,hypertension,heart_disease,avg_glucose_level,bmi,stroke,gender_Male,ever_married_Yes,work_type_Private,work_type_Self-employed,work_type_children,Residence_type_Urban,smoking_status_formerly smoked,smoking_status_never smoked,smoking_status_smokes
0,67.0,0,1,228.69,36.6,1,1,1,1,0,0,1,1,0,0
1,80.0,0,1,105.92,32.5,1,1,1,1,0,0,0,0,1,0


In [83]:
# set the target variable, i.e. the y column, and the x columns
ycol ="stroke"
num_cols = [x for x in df.columns if (x not in one_hot_cols) & (x != 'stroke')]
xcols = one_hot_cols + num_cols

In [84]:
#use Pandas to partition off a test set
idx = df.index
test_size = int(np.floor(len(idx) * 0.1))
test_idx = np.random.choice(idx, test_size)
test = df.loc[test_idx]
train_val = df.drop(test_idx, axis=0)

In [85]:
# use sklearn to partition train and test(really val)
# we could use sklearn for test as well
# but I find Pandas easer to use
X = train_val[xcols]
y = train_val[ycol]
Xtest = test[xcols]
ytest = test[ycol]
val_size = 0.2
Xtrain, Xval, ytrain, yval = train_test_split(X, y, test_size=val_size, random_state=42)

In [86]:
# define a dictional for the train, val and test sets, bot X and y for each
# and check the size of each
ds_dict = {"train" : (Xtrain, ytrain),
            "val" : (Xval, yval),
            "test": (Xtest, ytest)}
for ds in ds_dict.keys():
    X, y = ds_dict[ds]
    print(f"""{ds} X {X.shape} y {y.shape}""")

train X (3604, 14) y (3604,)
val X (901, 14) y (901,)
test X (498, 14) y (498,)


In [87]:
# define a function to create accuracy and loss
# for each of the datasets given an input model
def make_results(dsets, mod):
    dfs = []
    for name in dsets.keys():
        X, y  = dsets[name]
        rd = {}
        ypred = mod.predict(X)
        acc = accuracy_score(ypred, y.to_numpy().ravel())
        proba = mod.predict_proba(X)
        loss = log_loss(y.values, proba)
        score = mod.score(X,y)
        rd["acc"] = acc
        rd["loss"] = loss
        rdf = pd.DataFrame(rd, index=[0])
        rdf["ds"] = name
        dfs.append(rdf)
    res_df = pd.concat(dfs)    
    return res_df

In [103]:
# sklearn throws lots of warnings, espicially with Logistic Regression
# while not good practice in general, we will ignore the warnings
import warnings
def warn(*args, **kwargs):
    pass
#warnings.warn = warn

## Hyperparameter search
For all 3 methods we will use sklearn's GridSearchCV  
to search over a grid of hyper parameters and   
report on the results for the best

In [104]:
# Logistic Regression
from sklearn.linear_model import LogisticRegression

param_grid = {
    "penalty": ['l1', 'l2', 'elasticnet' ],
    "solver": [ "lbfgs", "saga" ],
    "max_iter": [200, 400, 600],
}
start = datetime.datetime.now()
grid_reg = GridSearchCV(
    LogisticRegression(), param_grid=param_grid,
    scoring="accuracy",
    refit=True,
    cv=5
)
regfit = grid_reg.fit(Xtrain, ytrain.to_numpy().ravel())

end = datetime.datetime.now()
print(f"search time {(end-start).total_seconds()}")

best_reg = regfit.best_estimator_
reg_res = make_results(dsets, mod=best_reg)
print(regfit.best_params_)
reg_res

{'max_iter': 200, 'penalty': 'l2', 'solver': 'lbfgs'}


,acc,loss,ds
0,0.950610,0.156825,train
0,0.948946,0.173313,val
0,0.953815,0.135607,test


## Logistic results
Note that the accuracy and loss are quit good for Logistic Regression  
which is perhaps surprising since this is the least sophisticated method.



In [109]:
# Gradient Boosting
from sklearn.ensemble import GradientBoostingClassifier


param_grid = {
    "n_estimators": [50, 100, 250, 500, 700 ],
    "max_leaf_nodes": [3, 4, 6],
    "learning_rate": [0.1, 0.05, 0.01],
}

start = datetime.datetime.now()
grid_gb = GridSearchCV(
    GradientBoostingClassifier(), param_grid=param_grid,
    scoring="accuracy",
    refit=True,
    cv=5
)

gfit = grid_gb.fit(Xtrain, ytrain.to_numpy().ravel())

end = datetime.datetime.now()
print(f"search time {(end-start).total_seconds()}")

best_gbm = gfit.best_estimator_
gbm_res = make_results(dsets, mod=best_gbm)
print(gfit.best_params_)
gbm_res

search time 132.604266
{'learning_rate': 0.1, 'max_leaf_nodes': 3, 'n_estimators': 50}


,acc,loss,ds
0,0.950888,0.151802,train
0,0.948946,0.168683,val
0,0.953815,0.136938,test


## Gradient Boosting results

Here the results are very similar, with the main difference in  
that the validation loss for GB is lower than for Logistic.   
But this is not really a fair comparison since the valididation set
is used in fitting with GB.

Also note that it takes quit a bit longer to do the search for GB.


In [114]:
# Random Forest
from sklearn.ensemble import RandomForestClassifier


param_grid = {
    "n_estimators": [50, 100, 250, 500, ],
    "max_depth": [2, 5, 10, 20],
     "min_samples_leaf": [1, 3, 5],
      "oob_score": ["True", "False"],
    "criterion" : ["gini", "entropy", "log_loss"]
    
}

start = datetime.datetime.now()
grid_rf = GridSearchCV(
    RandomForestClassifier(), param_grid=param_grid,
    scoring="accuracy",
    refit=True,
    cv=5
)
rf_fit = grid_rf.fit(Xtrain, ytrain.to_numpy().ravel())

end = datetime.datetime.now()
print(f"search time {(end-start).total_seconds()}")

best_rf = rf_fit.best_estimator_
rf_res = make_results(dsets, mod=best_rf)
print(rf_fit.best_params_)
rf_res

search time 607.084353
{'criterion': 'gini', 'max_depth': 10, 'min_samples_leaf': 1, 'n_estimators': 50, 'oob_score': 'True'}


,acc,loss,ds
0,0.964761,0.094885,train
0,0.958935,0.119844,val
0,0.959839,0.104992,test


## Random forest results
Clearly better than the previous 2, but maybe not fair since we searched  
over many more parameters.

Still, impressive on the test loss.